# Title: Universal_Pretrained_Model_Loader (TIMM)
## Description: `timm` 라이브러리를 사용하여 ResNet, EfficientNet, ViT 등 다양한 최신 모델을 불러오고, 마지막 분류기(Classifier)를 내 데이터의 클래스 개수에 맞게 자동으로 교체하는 팩토리 클래스.
## Input: 
 - model_name (str): timm에서 지원하는 모델명 (예: 'efficientnet_b0', 'resnet50')
 - num_classes (int): 분류할 클래스 개수
 - pretrained (bool): ImageNet 사전 학습 가중치 사용 여부 (Default: True)
## Output: 
 - model (nn.Module): 설정이 완료된 PyTorch 모델 객체
## Check Point: 
 - 인터넷 연결 필요 (최초 실행 시 가중치 다운로드).
 - 모델마다 입력 이미지 크기(Resolution) 요구사항이 다를 수 있음.

In [ ]:
import torch
import torch.nn as nn
import timm

class UniversalModel(nn.Module):
    """
    timm 라이브러리를 래핑(Wrapping)하여 다양한 백본을 손쉽게 교체할 수 있는 모델 클래스
    """
    def __init__(self, model_name, num_classes, pretrained=True, freeze_backbone=False):
        super(UniversalModel, self).__init__()
        
        # [Block 1] 모델 생성
        # timm.create_model은 다음 과정을 자동으로 수행합니다:
        # 1. 아키텍처 로드
        # 2. Pretrained Weight 로드 (pretrained=True 시)
        # 3. 마지막 FC Layer(Head)를 num_classes에 맞게 교체 및 초기화
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
        
        # [Block 2] (선택) Feature Extractor 동결 (Freezing)
        # 데이터가 너무 적거나, 사전 학습된 특징을 그대로 쓰고 싶을 때 사용
        if freeze_backbone:
            # 전체 파라미터를 얼리고
            for param in self.model.parameters():
                param.requires_grad = False
            
            # 마지막 분류기(Head)만 다시 녹임 (학습 가능하게)
            # timm 모델들은 보통 get_classifier() 메서드를 제공함
            for param in self.model.get_classifier().parameters():
                param.requires_grad = True

    def forward(self, x):
        return self.model(x)

# [Usage Example]
# model = UniversalModel(model_name='efficientnet_b0', num_classes=10)

## How to Use
1. **모델 이름 찾기**: 어떤 모델을 쓸지 모르겠다면 아래 코드로 검색해보세요.
    ```python
    import timm
    # 'efficientnet'이 포함된 모델명 리스트 출력
    print(timm.list_models('*efficientnet*')) 
    ```
2. **모델 선언**:
    ```python
    # 1위 코드에서 사용한 EfficientNet-B0 예시
    model = UniversalModel(model_name='efficientnet_b0', num_classes=10, pretrained=True)
    ```

## Troubleshooting
- **`RuntimeError: size mismatch`**: `pretrained=True`로 설정했는데 `num_classes`를 지정하지 않으면, 모델은 ImageNet 기준인 1000개 클래스를 출력합니다. 내 데이터의 라벨 개수와 맞지 않아 에러가 발생하므로 `num_classes`를 꼭 명시하세요.
- **입력 채널 에러**: 엑스레이나 흑백 이미지(1채널)를 사용할 경우, `in_chans=1` 인자를 `timm.create_model`에 추가로 전달해야 합니다. (위 코드를 수정하여 `**kwargs`를 받게 하면 더 유연해집니다.)